<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#About" data-toc-modified-id="About-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>About</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Data-import" data-toc-modified-id="Data-import-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data import</a></span><ul class="toc-item"><li><span><a href="#Data" data-toc-modified-id="Data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Data</a></span><ul class="toc-item"><li><span><a href="#train/test-data" data-toc-modified-id="train/test-data-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>train/test data</a></span></li><li><span><a href="#supplemental-data" data-toc-modified-id="supplemental-data-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>supplemental data</a></span></li><li><span><a href="#sample-submission" data-toc-modified-id="sample-submission-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>sample submission</a></span></li></ul></li></ul></li><li><span><a href="#Exploratory-data-analysis" data-toc-modified-id="Exploratory-data-analysis-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exploratory data analysis</a></span></li><li><span><a href="#model-training" data-toc-modified-id="model-training-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>model training</a></span><ul class="toc-item"><li><span><a href="#data-preprocessing" data-toc-modified-id="data-preprocessing-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>data preprocessing</a></span><ul class="toc-item"><li><span><a href="#aggregate-data-to-monthly-level" data-toc-modified-id="aggregate-data-to-monthly-level-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>aggregate data to monthly level</a></span></li><li><span><a href="#conversions,-clipping,-etc" data-toc-modified-id="conversions,-clipping,-etc-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>conversions, clipping, etc</a></span></li></ul></li><li><span><a href="#merge-with-supplemental-data" data-toc-modified-id="merge-with-supplemental-data-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>merge with supplemental data</a></span></li><li><span><a href="#add-features" data-toc-modified-id="add-features-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>add features</a></span><ul class="toc-item"><li><span><a href="#date-related-features-(not-used)" data-toc-modified-id="date-related-features-(not-used)-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>date related features (not used)</a></span></li><li><span><a href="#revenue-(not-used)" data-toc-modified-id="revenue-(not-used)-5.3.2"><span class="toc-item-num">5.3.2&nbsp;&nbsp;</span>revenue (not used)</a></span></li></ul></li><li><span><a href="#simple-model" data-toc-modified-id="simple-model-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>simple model</a></span><ul class="toc-item"><li><span><a href="#ideas" data-toc-modified-id="ideas-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>ideas</a></span></li><li><span><a href="#implementation" data-toc-modified-id="implementation-5.4.2"><span class="toc-item-num">5.4.2&nbsp;&nbsp;</span>implementation</a></span><ul class="toc-item"><li><span><a href="#model-per-item" data-toc-modified-id="model-per-item-5.4.2.1"><span class="toc-item-num">5.4.2.1&nbsp;&nbsp;</span>model per item</a></span></li><li><span><a href="#model-per-store" data-toc-modified-id="model-per-store-5.4.2.2"><span class="toc-item-num">5.4.2.2&nbsp;&nbsp;</span>model per store</a></span></li></ul></li><li><span><a href="#example" data-toc-modified-id="example-5.4.3"><span class="toc-item-num">5.4.3&nbsp;&nbsp;</span>example</a></span></li></ul></li></ul></li></ul></div>

In [223]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from itertools import product

import os

# About

Data Description

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.
File descriptions

    train.csv - the training set. Daily historical data from January 2013 to October 2015.
    test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
    sample_submission.csv - a sample submission file in the correct format.
    items.csv - supplemental information about the items/products.
    item_categories.csv  - supplemental information about the items categories.
    shops.csv- supplemental information about the shops.

Data fields

    ID - an Id that represents a (Shop, Item) tuple within the test set
    shop_id - unique identifier of a shop
    item_id - unique identifier of a product
    item_category_id - unique identifier of item category
    item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
    item_price - current price of an item
    date - date in format dd/mm/yyyy
    date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
    item_name - name of item
    shop_name - name of shop
    item_category_name - name of item category

This dataset is permitted to be used for any purpose, including commercial use.


# Setup

In [40]:
run_on_kaggle = False

if run_on_kaggle:
    input_path = '/kaggle/input/'
    data_path = input_path + "competitive-data-science-predict-future-sales/"
    output_path = ""
else:
    input_path = '../data/'
    data_path = input_path
    output_path = ""

In [83]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk(input_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

../data/items.csv
../data/item_categories.csv
../data/sales_train.csv
../data/sample_submission.csv
../data/shops.csv
../data/test.csv


# Data import

## Data

### train/test data

In [140]:
# train.csv - the training set. Daily historical data from January 2013 to October 2015.
sales_train = pd.read_csv(data_path + "sales_train.csv")

# test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
sales_test = pd.read_csv(data_path + "test.csv")

In [141]:
display(sales_train.head())
display(sales_test.head())

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


### supplemental data

In [44]:
# items.csv - supplemental information about the items/products.
items = pd.read_csv(data_path + "items.csv")

# item_categories.csv  - supplemental information about the items categories.
item_categories = pd.read_csv(data_path + "item_categories.csv")

# shops.csv- supplemental information about the shops.
shops = pd.read_csv(data_path + "shops.csv")

In [45]:
display(items.head())
display(item_categories.head())
display(shops.head())

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


### sample submission

Submissions are evaluated by root mean squared error (RMSE). True target values are clipped into [0,20] range.

In [46]:
# sample_submission.csv - a sample submission file in the correct format.
sample_submission = pd.read_csv(data_path + "sample_submission.csv")

In [47]:
sample_submission

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


# Exploratory data analysis

In [48]:
# keep a clean copy of the train data
train = sales_train.copy()

In [49]:
# convert to datetime 
train["date"] = pd.to_datetime(train["date"], format="%d.%m.%Y")

In [50]:
def calculate_revenue(df: pd.DataFrame) -> float:
    # some items are returned (item_cnt_day = -1.0); those need to be substracted
    return sum(df["item_price"]*df["item_cnt_day"])

In [51]:
revenue_per_shop_and_month = (
    train
    .groupby(["shop_id", "date_block_num"])
    .apply(lambda x: calculate_revenue(x))
    .rename("total_revenue_per_shop_and_month")
    .reset_index()
)

In [52]:
train = pd.merge(train, revenue_per_shop_and_month, how="left")

In [53]:
train[train["shop_id"] == 25]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,total_revenue_per_shop_and_month
1,2013-01-03,0,25,2552,899.00,1.0,5376478.13
2,2013-01-05,0,25,2552,899.00,-1.0,5376478.13
3,2013-01-06,0,25,2554,1709.05,1.0,5376478.13
4,2013-01-15,0,25,2555,1099.00,1.0,5376478.13
5,2013-01-10,0,25,2564,349.00,1.0,5376478.13
...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.00,1.0,7633958.00
2935845,2015-10-09,33,25,7460,299.00,1.0,7633958.00
2935846,2015-10-14,33,25,7459,349.00,1.0,7633958.00
2935847,2015-10-22,33,25,7440,299.00,1.0,7633958.00


In [54]:
train["item_id"].nunique()

21807

In [55]:
train["shop_id"].nunique()

60

In [56]:
train[(train["shop_id"] == 5) & (train["item_id"] == 5037)]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,total_revenue_per_shop_and_month
1953995,2014-09-21,20,5,5037,2599.0,1.0,1350828.02
2150561,2014-11-29,22,5,5037,2599.0,1.0,1750290.00
2288630,2014-12-28,23,5,5037,1999.0,1.0,2642538.00
2288631,2014-12-20,23,5,5037,1999.0,1.0,2642538.00
2335446,2015-01-02,24,5,5037,1999.0,1.0,1576480.00
2335447,2015-01-07,24,5,5037,1999.0,1.0,1576480.00
2618926,2015-05-29,28,5,5037,1299.0,1.0,1036892.00
2704068,2015-06-28,29,5,5037,1499.0,1.0,864974.00
2719247,2015-07-05,30,5,5037,1499.0,1.0,825422.00
2810661,2015-08-14,31,5,5037,1499.0,1.0,1160948.50


In [57]:
train["shop_id"].nunique() * train["item_id"].nunique()

1308420

In [58]:
train["shop_id"].nunique()

60

In [59]:
train["item_id"].nunique()

21807

In [60]:
sales_test["shop_id"].nunique()

42

In [61]:
sales_test["item_id"].nunique()

5100

In [62]:
sales_test[(sales_test["shop_id"] == 5) & (sales_test["item_id"] == 5037)]

,ID,shop_id,item_id
0,0,5,5037


In [63]:
sales_test["shop_id"].nunique() * sales_test["item_id"].nunique()

214200

In [64]:
pd.merge(sales_test, sample_submission, on="ID")

,ID,shop_id,item_id,item_cnt_month
0,0,5,5037,0.5
1,1,5,5320,0.5
2,2,5,5233,0.5
3,3,5,5232,0.5
4,4,5,5268,0.5
...,...,...,...,...
214195,214195,45,18454,0.5
214196,214196,45,16188,0.5
214197,214197,45,15757,0.5
214198,214198,45,19648,0.5


# model training

In [143]:
# keep a clean copy of the train + test data
train_raw = sales_train.copy()
test_raw = sales_test.copy()

## data preprocessing

### aggregate data to monthly level

In [111]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in train_raw['date_block_num'].unique():
    cur_shops = train_raw[train_raw['date_block_num'] == block_num]['shop_id'].unique()
    cur_items = train_raw[train_raw['date_block_num'] == block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])), dtype='int32'))

# turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

# get aggregated values for (shop_id, item_id, month)
item_cnt_month = (
    train_raw
    .groupby(["date_block_num", "shop_id", "item_id"])["item_cnt_day"]
    .sum()
    .rename("item_cnt_month")
    .reset_index()
)

# join aggregated data to the grid
train = pd.merge(grid, item_cnt_month, how='left', on=index_cols).fillna(0)

# sort the data
train = train.sort_values(['date_block_num','shop_id','item_id'])

### conversions, clipping, etc

In [112]:
# target values are clipped into [0,20] range, so we do the same
train['item_cnt_month'] = train['item_cnt_month'].clip(0,20)

In [129]:
train

,shop_id,item_id,date_block_num,item_cnt_month,item_name,item_category_id,item_price
0,0,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
1,1,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
2,2,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
3,3,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
4,4,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
...,...,...,...,...,...,...,...
10913845,55,22158,33,0.0,ЯМА (rem),40,249.0
10913846,56,22158,33,0.0,ЯМА (rem),40,249.0
10913847,57,22158,33,0.0,ЯМА (rem),40,249.0
10913848,58,22158,33,0.0,ЯМА (rem),40,249.0


## merge with supplemental data

In [114]:
items.head(2)

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76


In [115]:
train = pd.merge(train, items, on="item_id")

In [145]:
test = pd.merge(test_raw, items, on="item_id")

In [146]:
# we assume static prices for now
# TODO: use dynamic prices
item_id_price_df = train_raw[["item_id", "item_price"]].drop_duplicates()
item_id_to_price_dict = {row["item_id"]: row["item_price"] for _, row in item_id_price_df.iterrows()}

In [147]:
# add the item price
train["item_price"] = train["item_id"].map(item_id_to_price_dict)
test["item_price"] = test["item_id"].map(item_id_to_price_dict)

In [168]:
# deal with NaNs
train["item_price"] = train["item_price"].fillna(train["item_price"].median())
test["item_price"] = test["item_price"].fillna(test["item_price"].median())

In [116]:
item_categories.head(2)

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1


In [117]:
# item_category_name is probably not relevant
# train = pd.merge(train, item_categories, on="item_category_id")

In [118]:
shops.head(2)

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1


In [119]:
# shop name is probably not relevant
# train = pd.merge(train, shops, on="shop_id")

## add features

In [120]:
train

,shop_id,item_id,date_block_num,item_cnt_month,item_name,item_category_id
0,0,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40
1,1,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40
2,2,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40
3,3,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40
4,4,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40
...,...,...,...,...,...,...
10913845,55,22158,33,0.0,ЯМА (rem),40
10913846,56,22158,33,0.0,ЯМА (rem),40
10913847,57,22158,33,0.0,ЯМА (rem),40
10913848,58,22158,33,0.0,ЯМА (rem),40


### date related features (not used)

In [ ]:
# train["weekday"] = train["date"].dt.weekday

In [ ]:
# train["month"] = train["date"].dt.month

### revenue (not used)

In [131]:
# revenue_per_shop_and_month = (
#     train
#     .groupby(["shop_id", "date_block_num"])
#     .apply(lambda x: calculate_revenue(x))
#     .rename("total_revenue_per_shop_and_month")
#     .reset_index()
# )

In [ ]:
# train = pd.merge(train, revenue_per_shop_and_month, how="left")

In [ ]:
# revenue_per_item_and_month = (
#     train
#     .groupby(["item_id", "date_block_num"])
#     .apply(lambda x: calculate_revenue(x))
#     .rename("total_revenue_per_item_and_month")
#     .reset_index()
# )

In [ ]:
# train = pd.merge(train, revenue_per_item_and_month, how="left")

In [ ]:
# train

## simple model

### ideas

 * Hierachical time-series prediction:
  * model per store
  * model per item
  * model per date
* models to try:
 * Gradient Boosting Machine
 * XGBoost
* play around with lag
 * (df[NUMERIC_COLUMN] - df.groupby(DATE_COLUMN)[NUMERIC_COLUMN].shift(-1)) and agg features (df.groupby(DATE_COLUMN)[NUMERIC_COLUMN].agg(std))
* clip the test data!
 * predictions = numpy.clip(predictions, 0 ,20)

### implementation

In [224]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

#### model per item

In [226]:
def train_model(model, n_rows_to_train_on):
    
    train_subset = train[:n_rows_to_train_on].drop("item_name", axis=1)
    
    model.fit(X=train_subset.drop("item_cnt_month", axis=1), y=train_subset["item_cnt_month"])
    
    model_predictions = model_per_item.predict(test.drop("item_name", axis=1))
    model_predictions = model_predictions.clip(0, 20)
    
    return model, model_predictions

In [207]:
random_forest_reg = RandomForestRegressor(max_depth=4, random_state=0)

In [225]:
grad_boost_reg = GradientBoostingRegressor()

In [ ]:
random_forest_reg, random_forest_pred = train_model(random_forest_reg, n_rows_to_train_on=50000000)

In [228]:
grad_boost_reg, grad_boost_pred = train_model(grad_boost_reg, n_rows_to_train_on=50000000)

C:\Users\Telcontar\Anaconda3\envs\kaggle\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- ID
Feature names seen at fit time, yet now missing:
- date_block_num

  warnings.warn(message, FutureWarning)


In [230]:
grad_boost_pred

array([0.61923627, 0.61923627, 0.61923627, ..., 0.17625961, 0.17625961,
       0.17625961])

In [231]:
# write to submission file
submission_df = pd.DataFrame({"ID": test["ID"], "item_cnt_month": grad_boost_pred})

In [238]:
submission_df.to_csv("../output/submission.csv", index=False)

In [236]:
train["item_cnt_month"].mean()

0.2982399428249426

In [187]:
sample_submission

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [181]:
test

,ID,shop_id,item_id,item_name,item_category_id,item_price
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
1,5100,4,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
2,10200,6,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
3,15300,3,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
4,20400,2,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
...,...,...,...,...,...,...
214195,193799,46,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00
214196,198899,41,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00
214197,203999,44,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00
214198,209099,39,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00


In [180]:
sample_submission["item_cnt_month"] = 

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5
...,...,...
214195,214195,0.5
214196,214196,0.5
214197,214197,0.5
214198,214198,0.5


In [138]:
train

,shop_id,item_id,date_block_num,item_cnt_month,item_name,item_category_id,item_price
0,0,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
1,1,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
2,2,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
3,3,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
4,4,19,0,0.0,/ЗОЛОТАЯ КОЛЛЕКЦИЯ м/ф-72,40,28.0
...,...,...,...,...,...,...,...
10913845,55,22158,33,0.0,ЯМА (rem),40,249.0
10913846,56,22158,33,0.0,ЯМА (rem),40,249.0
10913847,57,22158,33,0.0,ЯМА (rem),40,249.0
10913848,58,22158,33,0.0,ЯМА (rem),40,249.0


In [148]:
# 
test

,ID,shop_id,item_id,item_name,item_category_id,item_price
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
1,5100,4,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
2,10200,6,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
3,15300,3,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
4,20400,2,5037,"NHL 15 [PS3, русские субтитры]",19,749.45
...,...,...,...,...,...,...
214195,193799,46,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00
214196,198899,41,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00
214197,203999,44,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00
214198,209099,39,969,3 ДНЯ НА УБИЙСТВО (BD),37,198.00


#### model per store

In [ ]:
model_per_store = RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
train

In [ ]:
# TODO: combine all data to monthly data (instead of daily data)

In [ ]:
n_rows_to_train_on = 50000

In [ ]:
model_per_store.fit(X=train.drop(["date", "item_cnt_month"], axis=1), y=train["item_cnt_month"])

In [ ]:
train

In [ ]:
test

In [ ]:
pd.merge(test, sample_submission, on="ID")

In [ ]:
regr.predict(test)

### example 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.datasets import make_regression

In [ ]:
regr = RandomForestRegressor(max_depth=2, random_state=0)

In [ ]:
X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)

In [ ]:
regr.fit(X, y)

In [ ]:
print(regr.predict([[0, 0, 0, 0]]))

In [ ]:
y

In [ ]:
X.shape

In [ ]:
X